In [2]:
%matplotlib
from IPython.html.services.config import ConfigManager
from IPython.paths import locate_profile
from IPython.display import Image

import sys
sys.path.append("..")
import statnlpbook.util as util
import statnlpbook.parsing as parsing
from statnlpbook.transition import *

cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
cm.update('livereveal', {
              'theme': 'solarized',
              'transition': 'slide',
              'start_slideshow_at': 'selected',
              'progress': 'true',
})

Using matplotlib backend: Qt5Agg


{'progress': 'true',
 'start_slideshow_at': 'selected',
 'theme': 'solarized',
 'transition': 'slide'}

<center>
<h2>Transition-based dependency parsing with dynamic oracles</h2>
</center>

### Dependency parsing ###

To represent the syntax of a sentence as directed labeled edges between words.
- Where labels represent dependencies between words.

The output graph should be:
- rooted
- acyclic
- signle-headed
- connected

### Dependency graph example ###

<img src="images/toBeAnimated/depParse1.png">

### Transition-based dependency parsing ###

Construct the dependency tree through a sequence of transition actions (label predictions).
- Each action, transforms the current state until a terminal state is reached.
- In essence, which label should we add next?
- Trained using a gold sequence of actions. 
  - Derived from an oracle (expert policy).

### Error propagation ###

The first error will confuse the classifier, since it moves the sequence to space not explored by the gold sequence of a actions.
- More errors will likely follow.

### How can Imitation Learning help with that? ### 

Imitation Learning addresses error propagation, by considering the interaction among the transition being considered and transitions to be predicted later in the sentence.
- explores the search space, but avoids enumerating all possible outputs.

<center>
<h2>Transition-based dependency parsing</h2>
</center>


### Configurations - states ###

Dependency graph
- $x = {0,1,...N}$ are the nodes, each of them representing one of the $N$ words in the sentence
- $a \subseteq x \times x \times L$ are labeled directed arcs between the words, with labels coming from a predefined set $L$.

### Configurations - states ###

- Stack $S$: a last-in, first-out memory to keep track of words to process later
- Buffer $B$: words not processed so far
- Arcs $A$: the dependency edges predicted so far

### Transition actions ###

$(S, i | B, A) > (S | i, B, A)$
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>shift</b>: push the word at the top of the buffer $B$ to the stack 

$(S | i, B, A) > (S, B, A)$
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>reduce</b>: pop the word at the top of the stack $S$ if it has a head 

$(S | i, j | B, A) > (S | i | j, B, A \cup (i, j, l))$
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>rightArc-label</b>: create a labeled arc from the word at the top of the stack $i$ to the word at the top of the buffer $j$

$(S | i, j | B, A) > (S, j | B, A \cup (j, i, l))$
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>leftArc-label</b>: create a labeled arc from the word at the top of the buffer $j$ to the word at the top of the stack $i$, if $i$ has no head

### Transition-based dependency parsing in action! ###

<img src="images/toBeAnimated/depParse1.png">

### Alternative transition systems ###

We just saw the Arc-Eager system (Nivre, 2003)
- Arc-Standard (Nivre, 2004)
- Easy-First (Goldberg and Elhadad, 2010)

Generally for transition systems:
- Actions should be easy to learn,
- with easier actions occuring earlier in the sequence.

<center>
<h2>Imitation Learning and Dynamic Oracles</h2>
</center>

### Training with exploration ###

Goldberg and Nivre (2013) proposed a training algorithm to learn from states that result from incorrect actions.
- i.e. states that cannot be reached by following the gold action sequence.

Mitigate the effects of error propagation.

This requires an expert policy that can return the optimal action for any given state.

### Is that DAgger? ###

Very similar to DAgger, except:
- The roll-in policy is parameterized by $k$ and $p$.
  - $k$ denotes the number of initial rounds where roll-in will be exclusively the expert policy \pi^{\star}.
  - $p$ is the probability of choosing the learned policy over the expert, at each time-step.
- We may have mutliple correct actions at each time-step.
  - Tie-breaks occur according to current prediction.

### Training formulation ###

<p style="border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 80%">
\begin{align}
& \textbf{Input:} \; D_{train} = \{(\mathbf{x}^1,\mathbf{y}^1)...(\mathbf{x}^M,\mathbf{y}^M)\}, \; expert\; \pi^{\star}, \; loss \; function \; \ell, parameters \; p, \; k\\
& \textbf{Output:} \; classifier \; H\\
& \; \\
& 1 \quad training\; examples\; \cal E = \emptyset\\
& 2 \quad \mathbf{while}\; \text{termination condition not reached}\; \mathbf{do}\\
& 3 \quad \quad \mathbf{if} \; i > k \; \mathbf{and} \; Rand() < p \; \mathbf{then}\\
& 3 \quad \quad \quad \text{set} \; rollin \; policy \; \pi^{in} = mix(H,\pi^{\star})\\
& 3 \quad \quad \mathbf{else}\\
& 3 \quad \quad \quad \text{set} \; rollin \; policy \; \pi^{in} = H\\
& 4 \quad \quad \mathbf{for} \; (\mathbf{x},\mathbf{y}) \in D_{train} \; \mathbf{do}\\
& 5 \quad \quad \quad \text{rollin to predict} \; \hat y_1\dots\hat y_N  = \pi^{in}(\mathbf{x})\\
& 6 \quad \quad \quad \mathbf{for} \; \hat y_n \in \hat y_1\dots\hat y_N \; \mathbf{do}\\
& 7 \quad \quad \quad \quad \text{ask} \pi^{in} \text{for best action}\; y^{\star} = \pi^{in}(\hat y_1\dots\hat y_{n-1}, \mathbf{x},\mathbf{y}) \; \\
& 8 \quad \quad \quad \quad \text{extract features}\; f=\phi(\mathbf{x},\hat y_1\dots \hat y_{n-1}) \\
& 9 \quad \quad \quad \quad \cal E = \cal E \cup (f,y^{\star})\\
& 10 \quad \quad \text{learn} \; H\; \text{from}\; \cal E\\
\end{align}
</p>

### Types of oracles (expert policies) ###


Static oracle
- A function mapping each state to an optimal action (to reach the optimal terminal state).
- Only defined for states reachable through the gold action sequence, undefined for all other states.
- Essentially, the static oracle is the gold action sequence itself.

Non-deterministic oracle
- A function mapping each state to a set of optimal actions.
- Only well-defined for states from which the optimal terminal state is still reachable.


### Dynamic oracle ###

Non-deterministic and complete oracle
- A function mapping each state to a set of optimal actions.
- Defined for all states.
  - Employs a cost function to measure the cost of the terminal state (if it is not optimal).
  - In this case, Hamming loss.

### Dynamic oracle example ###


<center>
<h2>Dependency parser experiments</h2>
</center>

### Effect of k and p ###
<img src="images/dependHeatMaps.png">

### Results ###
<img src="images/dependResults.png">

### Summary so far ### 

We discussed modifications to the DAgger framework.
- Hard decay schedule after $k$ epochs when determining the roll-in and roll-out policies.
- Using a mix of expert and learned policy during roll-outs.

We showed that dynamic oracles improves on the results of static orcales.